<a href="https://colab.research.google.com/github/Atomix77/IMLO-IP/blob/main/IMLO_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Data



In [14]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torchvision.models as models

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

trainingTransform = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.RandomHorizontalFlip(),
      transforms.RandomRotation(20),
      transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

testTransform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(torch.Tensor(mean), torch.Tensor(std)),])

trainingData = datasets.Flowers102(
    root = "./datasets",
    split = "train",
    transform = trainingTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

validationData = datasets.Flowers102(
    root = "./datasets",
    split = "val",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

testData = datasets.Flowers102(
    root = "./datasets",
    split = "test",
    transform = testTransform,
    # target_transform = Lambda(lambda y: torch.zeros(1000, dtype=torch.float).scatter_(0, torch.tensor(y), value=1)),
    download = True)

# Dataloaders

In [16]:
batchSize = 64

trainingDataloader = DataLoader(trainingData, batchSize, shuffle = True)
testDataloader = DataLoader(testData, batchSize)

# Get device

In [17]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


# Create Aritecture

In [18]:
class NeuralNetwork(nn.Module):
  def __init__(self, classAmount):
    super(NeuralNetwork, self).__init__()
    self.convStack =  nn.Sequential(
        nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Linear(128 * 28 * 28, classAmount)

  def forward(self, x):
    x = self.convStack(x)
    x = torch.flatten(x, 1)
    x = self.classifier(x)
    return x

model = NeuralNetwork(102).to(device)

# Training

In [19]:
def trainingLoop(dataloader, model, lossFunction, optimizer):
  size = len(dataloader.dataset)
  model.train()

  for batch, (X, Y) in enumerate(dataloader):
    pred = model(X)
    loss = lossFunction(pred, Y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch % 100 == 0:
      loss, current = loss.item(), batch * batchSize + len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [20]:
def testingLoop(dataloader, model, lossFunction):
  model.eval()
  size = len(dataloader.dataset)
  numBatches = len(dataloader)
  testLoss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      testLoss += lossFunction(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    testLoss /= numBatches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {testLoss:>8f} \n")

In [21]:
learningRate = 1e-3
batchSize = 64
epochs = 5

lossFunction = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)

for i in range(epochs):
  print(f"Epoch {i+1}\n-------------------------------")
  trainingLoop(trainingDataloader, model, lossFunction, optimizer)
  testingLoop(testDataloader, model, lossFunction)
print("Finished")

Epoch 1
-------------------------------
loss: 4.638793  [   64/ 1020]
Test Error: 
 Accuracy: 0.6%, Avg loss: 4.617496 

Epoch 2
-------------------------------
loss: 4.618838  [   64/ 1020]


KeyboardInterrupt: 

# Saving the model

In [ ]:
# model = models.vgg16(weights='IMAGENET1K_V1')
# torch.save(model, 'model.pth')

# Loading the model

In [ ]:
# model = models.vgg16()
# model.load_state_dict(torch.load('model.pth'))
# model.eval()